In [1]:
#raspa tela
import requests
from bs4 import BeautifulSoup
#dataframe
import pandas as pd
#gráficos
import matplotlib.pyplot as plt
#data e hora.
import datetime as dtm
from datetime import date as dt
import time
import re
import numpy as np
from unidecode import unidecode
from datetime import datetime



# Informando período

In [2]:
#definir quais anos você quer analisar
AnoInicial=int(input("Informe o Ano inicial: "))
AnoFinal=int(input("Informe o Ano final: "))
print("Lembrando que quanto mais anos, mais demorado será a extração")

Informe o Ano inicial:  2018
Informe o Ano final:  2023


Lembrando que quanto mais anos, mais demorado será a extração


# Processo de Web Scraping

In [3]:
tempo_inicio = time.time()

linkRaiz = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a'
listaLinks = []
ListaFinal = []
jogo=[]
Info = []
Rodada = []
j = []
x = []

#Definindo Range e iniciando o BeautifulSoup
for ano in range(AnoInicial, AnoFinal+1):
    for nj in range(1,381):
        linkJogo = f"{linkRaiz}/{ano}/{nj}"
        listaLinks.append(linkJogo)
        valorweb = requests.get(linkJogo)
        soup = BeautifulSoup(valorweb.content, 'html.parser')

#Pegando informações referente ao jogo e adicionando as listas  
        title = soup.title.text
        jogos=soup.find(class_='color-white block text-1').text
        jogo.append(jogos)
        ListaFinal.append(title)
        #pegando informações sobre o Local do jogo
        localdthr_jogo = soup.find_all(class_='text-2 p-r-20')
        Texto = [element.get_text(strip=True) for element in localdthr_jogo]
        Info.append(Texto)
        #Numerojogo
        jogos=soup.find(class_='color-white block text-1').text
        jogos = jogos.split(':')[1].strip()
        
        #Lista de gol por jogador
        GolJog = soup.find_all(class_='hidden-sm hidden-md hidden-lg m-t-20')
        i = [element.get_text(strip=True) for element in GolJog]
        j.extend(i)

        #lista de gol por time
        captTimes = soup.find_all(class_='time-nome color-white')
        Texto = [element.get_text(strip=True) for element in captTimes]
        x.extend(Texto)
           
#Matriz de Gols de jogadores por time
matriz = [[i_item, Texto_item] for i_item, Texto_item in zip(j,x) if j and x != '']

#tempo de execução
tempo_final = time.time()
tempo_decorrido = tempo_final - tempo_inicio


print(f'Tempo decorrido: {tempo_decorrido:.2f} segundos')

Tempo decorrido: 1852.33 segundos


# Iniciar o Processo de DataFrame para acompanhamento dos Jogos

In [4]:
Estadio = []
Cidade = []
Estado = []
Data = []
Horario = []
for item in Info:
    if len(item)>2:
            texto = item[0]  # O nome do estádio está na primeira posição de cada item
            padrao_estadio = r'^(.*?) -'
            resultado_estadio = re.search(padrao_estadio, texto)
            if resultado_estadio:
                nome_estadio = resultado_estadio.group(1)
                Estadio.append(nome_estadio)
            padrao_cidade = r'- (.*?) -'
            resultado_cidade = re.search(padrao_cidade, texto)
            if resultado_cidade:
                nome_cidade = resultado_cidade.group(1)
                Cidade.append(nome_cidade)
            padrao_estado = r'- ([A-Z]{2})$'
            resultado_estado = re.search(padrao_estado, texto)
            if resultado_estado:
                estado = resultado_estado.group(1)
                Estado.append(estado)
            texto1 = item[1]
            padrao_data = r'\w+, \d+ de \w+ de \d+'
            datas_encontradas = re.search(padrao_data, texto1)
            if datas_encontradas:
                data = datas_encontradas.group(0)
                Data.append(data)
    else:
        Estadio.append("Não definido")
        Cidade.append("Não definido")
        Estado.append("Não definido")
        Data.append("Não definido")
        
for item in Info:
    if len(item) > 2:
        horario = item[2]
        if horario != "A definir":
            Horario.append(horario)
        else:
            Horario.append("Não definido")
    else:
        Horario.append("Não definido")
len(Horario)

2280

In [5]:
#Coletar Número Jogo
NumeroJogo=[]
for i in jogo:
    j=i.split('Jogo:')[1].strip()
    NumeroJogo.append(j)


#Coletar Time da Casa
time_casa = []
UF_Time_Mandante = []
for partidas in ListaFinal:
    times = partidas.split(' x ')
    timeCasa = times[0].strip()  # Remover espaços extras no início e no final
    nome_time_casa = timeCasa.split(' - ')[0].strip()  # Separar o nome do time da parte do estado
    Estado= timeCasa.split(' - ')[1].strip()
    Estado = Estado.split(' ')[0]
    time_casa.append(nome_time_casa )
    UF_Time_Mandante.append(Estado)


#coletar Visitante
time_visitante = []
UF_Time_Visitante = []
for partidas in ListaFinal:
    times = partidas.split(' x ')
    timeVisitante = times[1].strip()  # Remover espaços extras no início e no final
    nome_time_visitante = timeVisitante.split(' - ')[0].strip() 
    nome_time_visitante1 = nome_time_visitante[2:]  # Remover o número no início
    time_visitante.append(nome_time_visitante1)
    Estado = timeVisitante.split('-')[1].strip()
    UF_Time_Visitante.append(Estado)



#coletar gols mandante
gols_mandante = []

for gols in ListaFinal:
    gol_mandante = gols.split(' x ')[0].strip()  # Extrair os gols do time da casa
    time = gol_mandante.split(' - ')[1].strip()
    time=time[3:]
    gols_mandante.append(time)

#Coletar gols visitante
gols_visitante=[]
for gols in ListaFinal:
    gols_visitantes=gols.split(' x ')[1].strip()
    gols_visitantes=gols_visitantes.split(' - ')[0].strip()
    gols_visitantes=gols_visitantes.split(' ')[0]
    gols_visitante.append(gols_visitantes)

#Coletar Ano
anoJogo=[]
for i in ListaFinal:
    Ano_Jogos=i.split(' x ')[1].strip()
    Ano_Jogos=Ano_Jogos.split(' - ')[3]
    Ano_Jogos=Ano_Jogos[-4:]
    anoJogo.append(Ano_Jogos)

In [6]:
#Determinar quem venceu
def vitoria(gols_mandante, gols_visitante):
    if gols_mandante > gols_visitante:
        return 'Mandante'
    elif gols_mandante < gols_visitante:
        return 'Visitante'
    else:
        return 'Empate'
resultado=[vitoria(gols_mandante,gols_visitante) for gols_mandante,gols_visitante in zip(gols_mandante,gols_visitante)]

In [7]:
#Criando o DataFrame 
data = {'NumeroJogo':NumeroJogo,'Data':Data,'Hora':Horario,'Estadio':Estadio,'Cidade':Cidade,'UF':Estado, 'Quem Venceu?':resultado, 'AnoJogo':anoJogo, 'Time Mandante':time_casa,'UF Mandante': UF_Time_Mandante , 'Gols Mandante':gols_mandante,'Time Visitante':time_visitante,'UF Visitante': UF_Time_Visitante,'Gols Visitante':gols_visitante}
df = pd.DataFrame(data)
df

,NumeroJogo,Data,Hora,Estadio,Cidade,UF,Quem Venceu?,AnoJogo,Time Mandante,UF Mandante,Gols Mandante,Time Visitante,UF Visitante,Gols Visitante
0,1,"Sábado, 14 de Abril de 2018",16:00,Mineirão,Belo Horizonte,PR,Visitante,2018,Cruzeiro,MG,0,Grêmio,RS,1
1,2,"Domingo, 15 de Abril de 2018",19:00,Arena da Baixada,Curitiba,PR,Mandante,2018,Athletico Paranaense,PR,5,Chapecoense,SC,1
2,3,"Domingo, 15 de Abril de 2018",11:00,Independência,Belo Horizonte,PR,Mandante,2018,America Fc,MG,3,Sport,PE,0
3,4,"Sábado, 14 de Abril de 2018",19:00,Manoel Barradas,Salvador,PR,Empate,2018,Vitória,BA,2,Flamengo,RJ,2
4,5,"Domingo, 15 de Abril de 2018",16:00,São Januário,Rio de Janeiro,PR,Mandante,2018,Vasco da Gama,RJ,2,Atlético,MG,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,376,"Quarta, 06 de Dezembro de 2023",21:30,Mineirão,Belo Horizonte,PR,Empate,2023,Cruzeiro Saf,MG,1,Palmeiras,SP,1
2276,377,"Quarta, 06 de Dezembro de 2023",21:30,Beira-Rio,Porto Alegre,PR,Mandante,2023,Internacional,RS,3,Botafogo,RJ,1
2277,378,"Quarta, 06 de Dezembro de 2023",21:30,Couto Pereira,Curitiba,PR,Visitante,2023,Coritiba S.a.f.,PR,0,Corinthians,SP,2
2278,379,"Quarta, 06 de Dezembro de 2023",21:30,Arena Fonte Nova,Salvador,PR,Mandante,2023,Bahia,BA,4,Atlético Mineiro Saf,MG,1


In [8]:
#Excluindo jogos que ainda não ocorreram 
df['Gols Visitante'] = pd.to_numeric(df['Gols Visitante'], errors='coerce').astype('Int64')
df['Gols Mandante'] = pd.to_numeric(df['Gols Mandante'], errors='coerce').astype('Int64')
df['NumeroJogo'] = pd.to_numeric(df['NumeroJogo'], errors='coerce').astype('Int64')
df['Rodada'] = df['NumeroJogo'].apply(lambda num_jogo: int(num_jogo / 10) if num_jogo % 10 == 0 else int(num_jogo // 10) + 1)
df['Time Mandante'] = df['Time Mandante'].str.replace('.', '')
df['Time Visitante'] = df['Time Visitante'].str.replace('.', '')
df = df.dropna(subset=['Gols Visitante'])
X = ['NumeroJogo', 'Data','Hora','Estadio', 'Cidade','UF','Quem Venceu?', 'AnoJogo', 'Rodada', 'Time Mandante', 'UF Mandante', 'Gols Mandante', 'Time Visitante', 'UF Visitante', 'Gols Visitante']
df = df[X]
df

,NumeroJogo,Data,Hora,Estadio,Cidade,UF,Quem Venceu?,AnoJogo,Rodada,Time Mandante,UF Mandante,Gols Mandante,Time Visitante,UF Visitante,Gols Visitante
0,1,"Sábado, 14 de Abril de 2018",16:00,Mineirão,Belo Horizonte,PR,Visitante,2018,1,Cruzeiro,MG,0,Grêmio,RS,1
1,2,"Domingo, 15 de Abril de 2018",19:00,Arena da Baixada,Curitiba,PR,Mandante,2018,1,Athletico Paranaense,PR,5,Chapecoense,SC,1
2,3,"Domingo, 15 de Abril de 2018",11:00,Independência,Belo Horizonte,PR,Mandante,2018,1,America Fc,MG,3,Sport,PE,0
3,4,"Sábado, 14 de Abril de 2018",19:00,Manoel Barradas,Salvador,PR,Empate,2018,1,Vitória,BA,2,Flamengo,RJ,2
4,5,"Domingo, 15 de Abril de 2018",16:00,São Januário,Rio de Janeiro,PR,Mandante,2018,1,Vasco da Gama,RJ,2,Atlético,MG,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,376,"Quarta, 06 de Dezembro de 2023",21:30,Mineirão,Belo Horizonte,PR,Empate,2023,38,Cruzeiro Saf,MG,1,Palmeiras,SP,1
2276,377,"Quarta, 06 de Dezembro de 2023",21:30,Beira-Rio,Porto Alegre,PR,Mandante,2023,38,Internacional,RS,3,Botafogo,RJ,1
2277,378,"Quarta, 06 de Dezembro de 2023",21:30,Couto Pereira,Curitiba,PR,Visitante,2023,38,Coritiba Saf,PR,0,Corinthians,SP,2
2278,379,"Quarta, 06 de Dezembro de 2023",21:30,Arena Fonte Nova,Salvador,PR,Mandante,2023,38,Bahia,BA,4,Atlético Mineiro Saf,MG,1


### Acompanhar desempenho dos times Individualmente

In [9]:
#Copiar Dataframe anterior
dx_local = df[['NumeroJogo', 'Time Mandante','UF Mandante', 'Gols Mandante', 'Quem Venceu?','Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo', 'Gols Visitante']].copy()
dx_visitante = df[['NumeroJogo', 'Time Visitante','UF Visitante', 'Gols Visitante', 'Quem Venceu?', 'Estadio', 'Cidade','UF', 'Hora', 'Data','AnoJogo', 'Gols Mandante']].copy()
dx_local.columns = ['NumeroJogo', 'Time','UF Time', 'Gols Feitos', 'Quem Venceu?','Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo', 'Gols Sofrido']
dx_local['Mandante/Visitante'] = 'Mandante'
dx_visitante.columns = ['NumeroJogo', 'Time','UF Time', 'Gols Feitos', 'Quem Venceu?', 'Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo', 'Gols Sofrido']
dx_visitante['Mandante/Visitante'] = 'Visitante'
#concatenar
dx = pd.concat([dx_local, dx_visitante], ignore_index=True)
#Demonstrar pontuação
dx['Pontos'] = dx.apply(lambda row: 3 if row['Quem Venceu?'] == row['Mandante/Visitante'] else (1 if row['Quem Venceu?'] == 'Empate' else 0), axis=1)
dx = dx.drop('Quem Venceu?', axis=1)
dx

,NumeroJogo,Time,UF Time,Gols Feitos,Estadio,Cidade,UF,Hora,Data,AnoJogo,Gols Sofrido,Mandante/Visitante,Pontos
0,1,Cruzeiro,MG,0,Mineirão,Belo Horizonte,PR,16:00,"Sábado, 14 de Abril de 2018",2018,1,Mandante,0
1,2,Athletico Paranaense,PR,5,Arena da Baixada,Curitiba,PR,19:00,"Domingo, 15 de Abril de 2018",2018,1,Mandante,3
2,3,America Fc,MG,3,Independência,Belo Horizonte,PR,11:00,"Domingo, 15 de Abril de 2018",2018,0,Mandante,3
3,4,Vitória,BA,2,Manoel Barradas,Salvador,PR,19:00,"Sábado, 14 de Abril de 2018",2018,2,Mandante,1
4,5,Vasco da Gama,RJ,2,São Januário,Rio de Janeiro,PR,16:00,"Domingo, 15 de Abril de 2018",2018,1,Mandante,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555,376,Palmeiras,SP,1,Mineirão,Belo Horizonte,PR,21:30,"Quarta, 06 de Dezembro de 2023",2023,1,Visitante,1
4556,377,Botafogo,RJ,1,Beira-Rio,Porto Alegre,PR,21:30,"Quarta, 06 de Dezembro de 2023",2023,3,Visitante,0
4557,378,Corinthians,SP,2,Couto Pereira,Curitiba,PR,21:30,"Quarta, 06 de Dezembro de 2023",2023,0,Visitante,3
4558,379,Atlético Mineiro Saf,MG,1,Arena Fonte Nova,Salvador,PR,21:30,"Quarta, 06 de Dezembro de 2023",2023,4,Visitante,0


In [10]:
#Tratamento dos Times
dx['Time'] = dx['Time'].apply(unidecode)
dx['Time'] = dx['Time'].str.replace(r'\s*S\.?A\.?F\s*', '', regex=True, case=False)
dx['Time'] = dx['Time'].apply(str.title)
dx['Time'] = dx['Time'].apply(lambda x: x.replace('Fc', '').strip())  # Remove "Fc" e espaços antes e depois
# Aplicar a lambda para modificar o valor da coluna 'Time' com base nas condições
dx['Time'] = dx.apply(lambda row: 'Athletico Paranaense' if row['Time'] == 'Atletico' and row['UF Time'] == 'PR' else row['Time'],axis=1)
dx['Time'] = dx.apply(lambda row: 'America Mineiro' if row['Time'] == 'America' and row['UF Time'] == 'MG' else row['Time'],axis=1)
dx['Time'] = dx.apply(lambda row: 'Atletico Mineiro' if row['Time'] == 'Atletico' and row['UF Time'] == 'MG' else row['Time'],axis=1)
dx['Time'] = dx.apply(lambda row: 'Atletico Goianiense' if row['Time'] == 'Atletico' and row['UF Time'] == 'GO' else row['Time'],axis=1)
dx['Rodada'] = dx['NumeroJogo'].apply(lambda num_jogo: int(num_jogo / 10) if num_jogo % 10 == 0 else int(num_jogo // 10) + 1)
dx['Resultado'] = dx.apply(lambda row: 'Vitória' if row['Pontos'] == 3 else ('Empatou' if row['Pontos'] == 1 else 'Derrota'), axis=1)

dx


,NumeroJogo,Time,UF Time,Gols Feitos,Estadio,Cidade,UF,Hora,Data,AnoJogo,Gols Sofrido,Mandante/Visitante,Pontos,Rodada,Resultado
0,1,Cruzeiro,MG,0,Mineirão,Belo Horizonte,PR,16:00,"Sábado, 14 de Abril de 2018",2018,1,Mandante,0,1,Derrota
1,2,Athletico Paranaense,PR,5,Arena da Baixada,Curitiba,PR,19:00,"Domingo, 15 de Abril de 2018",2018,1,Mandante,3,1,Vitória
2,3,America Mineiro,MG,3,Independência,Belo Horizonte,PR,11:00,"Domingo, 15 de Abril de 2018",2018,0,Mandante,3,1,Vitória
3,4,Vitoria,BA,2,Manoel Barradas,Salvador,PR,19:00,"Sábado, 14 de Abril de 2018",2018,2,Mandante,1,1,Empatou
4,5,Vasco Da Gama,RJ,2,São Januário,Rio de Janeiro,PR,16:00,"Domingo, 15 de Abril de 2018",2018,1,Mandante,3,1,Vitória
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555,376,Palmeiras,SP,1,Mineirão,Belo Horizonte,PR,21:30,"Quarta, 06 de Dezembro de 2023",2023,1,Visitante,1,38,Empatou
4556,377,Botafogo,RJ,1,Beira-Rio,Porto Alegre,PR,21:30,"Quarta, 06 de Dezembro de 2023",2023,3,Visitante,0,38,Derrota
4557,378,Corinthians,SP,2,Couto Pereira,Curitiba,PR,21:30,"Quarta, 06 de Dezembro de 2023",2023,0,Visitante,3,38,Vitória
4558,379,Atletico Mineiro,MG,1,Arena Fonte Nova,Salvador,PR,21:30,"Quarta, 06 de Dezembro de 2023",2023,4,Visitante,0,38,Derrota


### Classificação dos times

In [11]:
# Group by 'Time' and 'AnoJogo', calculate the sum of specified columns, and reset the index
classificacao = dx.groupby(['Time', 'AnoJogo']).agg({'Pontos': 'sum', 'Gols Feitos': 'sum', 'Gols Sofrido': 'sum'}).reset_index()

# Calculate 'Saldo' (goal difference) for each row
classificacao['Saldo'] = classificacao['Gols Feitos'] - classificacao['Gols Sofrido']

# Calcule 'Vitórias, empates e derrotas' para cada time e ano
vitorias = dx.loc[dx['Pontos'] == 3].groupby(['Time', 'AnoJogo']).size().reset_index(name='Vitorias')
derrotas = dx.loc[dx['Pontos'] == 0].groupby(['Time', 'AnoJogo']).size().reset_index(name='Derrotas')
empates = dx.loc[dx['Pontos'] == 1].groupby(['Time', 'AnoJogo']).size().reset_index(name='Empates')


# Juntando 'Vitorias' com 'classificacao'
classificacao = pd.merge(classificacao, vitorias, on=['Time', 'AnoJogo'], how='left')

# Juntando 'Derrotas' com 'classificacao'
classificacao = pd.merge(classificacao, derrotas, on=['Time', 'AnoJogo'], how='left')

# Juntando 'Empates' com 'classificacao'
classificacao = pd.merge(classificacao, empates, on=['Time', 'AnoJogo'], how='left')


classificacao.rename(columns={'AnoJogo': 'Temporada'}, inplace=True)

# Preencha valores NaN com 0
classificacao['Vitorias'] = classificacao['Vitorias'].fillna(0).astype(int)
classificacao['Derrotas'] = classificacao['Derrotas'].fillna(0).astype(int)
classificacao['Empates'] = classificacao['Empates'].fillna(0).astype(int)
classificacao['Pontos'] = classificacao['Pontos'].fillna(0).astype(int)
classificacao['Temporada'] = classificacao['Temporada'].fillna(0).astype(int)
classificacao = classificacao.sort_values(['Temporada', 'Pontos'])
classificacao

,Time,Temporada,Pontos,Gols Feitos,Gols Sofrido,Saldo,Vitorias,Derrotas,Empates
95,Parana,2018,23,18,57,-39,4,23,11
119,Vitoria,2018,37,36,63,-27,9,19,10
0,America Mineiro,2018,40,30,47,-17,10,18,10
112,Sport,2018,42,35,57,-22,11,18,9
115,Vasco Da Gama,2018,43,41,48,-7,10,15,13
...,...,...,...,...,...,...,...,...,...
30,Botafogo,2023,64,58,37,21,18,10,10
18,Atletico Mineiro,2023,66,52,32,20,19,10,9
60,Flamengo,2023,66,56,42,14,19,10,9
80,Gremio,2023,68,63,56,7,21,12,5


## Informação de Gols por jogador e time

### Alguns jogos os times não liberaram a escalação, então falta informação no site da CBF e consequentemente faltaram alguns gols dos atletas
#### ex: jogo 5 (2023) - RB Bragantino VS Bahia 

In [12]:
#Criando matriz para iterar jogos, times, jogador e jogos
x = np.array(matriz)
y = np.array(NumeroJogo)
j = np.array(anoJogo)
matriz1_reshaped = x.reshape(-1, 2, 2)

resultado = []

# Use um índice para rastrear o número do jogo da matriz 2
indice_matriz2 = 0

# Itere sobre a matriz 1 agrupando os registros em jogos e adicionando o número do jogo

#pegando pares de elementos da lista x, combinando-os com informações de j e y e em seguida, adicionando essas informações a uma lista resultado
for i in range(0, len(x), 2):
    jogo = x[i:i + 2]
    ano = j[indice_matriz2]  
    numero_jogo = y[indice_matriz2]
    resultado.append([numero_jogo, jogo, ano]) 
    indice_matriz2 += 1


numeros_jogo = []
detalhes_jogo = []
anos = []

# Itere sobre o resultado
for item in resultado:
    numeros_jogo.append(item[0])
    detalhes_jogo.append(item[1])
    anos.append(item[2])


In [13]:
lista_de_listas = []
# Itere pelos dados originais e crie uma nova lista para cada elemento
for elemento in detalhes_jogo:
    nova_lista = [item for sublist in elemento for item in sublist]
    lista_de_listas.append(nova_lista)
lista_de_listas = list(zip(numeros_jogo, lista_de_listas, anos))



In [14]:
pd_teste = pd.DataFrame({'testando':lista_de_listas})
pd_teste.to_excel(f"C:/Users/aliss/OneDrive/Área de Trabalho/NovoSistema/valor.xlsx")

In [15]:
detalhe = []
jogo = []
ano=[]

for elemento in lista_de_listas:
    numero_jogo = elemento[0]
    gols_detalhes = elemento[1]
    anos=elemento[2]
    detalhe.extend(gols_detalhes)
   
    contagem_t = 0
    for texto in gols_detalhes:
        contagem_t += texto.count('T)')
    
    jogo.extend([numero_jogo] * contagem_t)
    ano.extend([anos] * contagem_t)

nomes_jogadores = []
nomes_times = []

for i, dado in enumerate(detalhe):
    
    if i % 2 == 0:
        nomes = dado.split(')')
        nomes = [nome.strip() + ')' for nome in nomes if nome.strip()]
        nomes_jogadores.extend(nomes)
    else:
        # Multiplicar o nome do time pelo número de jogadores
        nomes_times.extend([dado] * len(nomes))
times = []
uf = []
for i in nomes_times:
    j = i.split('-')
    times.append(j[0])
    uf.append(j[1].strip())


In [16]:
#Criando Regex através de Ferramenta de identificação de padrões 
padrao = r'([\w\s]+)\s(\d{1,2}\+?\d*)\'\s\((\d{1,2}[+-]?\d*\w*)\)'
# Lista para armazenar os resultados
resultados = []
Atleta = []
Minutos = []
Tempo = []
Minutos_Acrescido = []

# Loop sobre as strings
for s in nomes_jogadores:
    correspondencias = re.search(padrao, s)
    if correspondencias:
        nome = correspondencias.group(1)
        minuto = correspondencias.group(2)
        tempo = correspondencias.group(3)
        resultados.append({'Nome': nome, 'Minuto': minuto, 'Tempo': tempo})
        Atleta.append(nome)
        Minutos.append(minuto)
        Tempo.append(tempo)
nomeOrientado=[]
for i in Atleta:
    x = i.capitalize()
    nomeOrientado.append(x)
    
    

for i in Minutos:
    x = i.split('+')
    z = int(x[0])
    
    for acrescimo in x[1:]:
        z += int(acrescimo)

    Minutos_Acrescido.append(z)

nomeOrientado


dy = pd.DataFrame({
    'Nome': nomeOrientado,
    'Minuto': Minutos_Acrescido,
    'Tempo': Tempo,
    'Time': times,
    'UF': uf,
    'Jogo': jogo,
    'Ano': ano
})

In [17]:
dy['Jogo'] = pd.to_numeric(dy['Jogo'], errors='coerce').astype('Int64')
dy['Time'] = dy['Time'].apply(unidecode)
dy['Time'] = dy['Time'].str.replace(r'\s*S\.?A\.?F\s*', '', regex=True, case=False)  # Remove SAF
dy['Time'] = dy['Time'].apply(str.title)
dy['Nome'] = dy['Nome'].apply(str.title)
dy['Time'] = dy['Time'].apply(lambda x: x.replace('Fc', '').strip())  # Remove "Fc" e espaços antes e depois
dy['Minuto'] = pd.to_numeric(dy['Minuto'], errors='coerce')
dy['Minuto'] = dy.apply(lambda row: row['Minuto'] + 45 if row['Tempo'] == '2ºT' else row['Minuto'], axis=1)
dy = dy.drop('Tempo', axis=1)
dy['Rodada'] = dy['Jogo'].apply(lambda num_jogo: int(num_jogo / 10) if num_jogo % 10 == 0 else int(num_jogo // 10) + 1)
dy['Time'] = dy.apply(lambda row: 'Athletico Paranaense' if row['Time'] == 'Atletico' and row['UF'] == 'PR' else row['Time'],axis=1)
dy['Time'] = dy.apply(lambda row: 'America Mineiro' if row['Time'] == 'America' and row['UF'] == 'MG' else row['Time'],axis=1)
dy['Time'] = dy.apply(lambda row: 'Atletico Mineiro' if row['Time'] == 'Atletico' and row['UF'] == 'MG' else row['Time'],axis=1)
dy['Time'] = dy.apply(lambda row: 'Atletico Goianiense' if row['Time'] == 'Atletico' and row['UF'] == 'GO' else row['Time'],axis=1)
dy['Time'] = dy['Time'].str.replace('.', '')
dy = dy.drop('UF',axis=1)
dy

,Nome,Minuto,Time,Jogo,Ano,Rodada
0,Andre Felipe,54,Gremio,1,2018,1
1,Pablo,51,Athletico Paranaense,2,2018,1
2,Nikao,58,Athletico Paranaense,2,2018,1
3,Carleto,68,Athletico Paranaense,2,2018,1
4,Rossetto,87,Athletico Paranaense,2,2018,1
...,...,...,...,...,...,...
5293,Ademir,92,Bahia,379,2023,38
5294,Paulinho,35,Atletico Mineiro,379,2023,38
5295,Clayson,12,Cuiaba,380,2023,38
5296,Derik,16,Cuiaba,380,2023,38


In [18]:
#Exportando todos Dataframe para Excel
with pd.ExcelWriter(f"C:/Users/aliss/OneDrive/Área de Trabalho/NovoSistema/CampeonatoBrasileiro_{AnoInicial}-{AnoFinal}_SerieA.xlsx") as writer:
    # Escreva cada DataFrame em uma planilha diferente
    df.to_excel(writer, sheet_name='Jogos', index=False)
    dx.to_excel(writer, sheet_name='DesempenhoIndividual', index=False)
    classificacao.to_excel(writer, sheet_name='DesempenhoTimes', index=False)
    dy.to_excel(writer, sheet_name='Gols', index=False)
